## Limpieza de Dataset de violencia intrafamiliar

In [2]:
import pandas as pd

df_vio = pd.read_csv("files/Violencia_Intrafamiliar_Limpio.csv", sep=";", dtype=str)

FECHAS

In [3]:
columnas_enteras = ["ANO", "DIA", "VIC_EDAD", "TOTAL_HIJOS", "AGR_EDAD"]

for col in columnas_enteras:
    df_vio[col] = pd.to_numeric(df_vio[col], errors="coerce").astype("Int64")

EDADES

In [4]:
df_vio["VIC_EDAD"] = pd.to_numeric(df_vio["VIC_EDAD"], errors="coerce")
df_vio["AGR_EDAD"] = pd.to_numeric(df_vio["AGR_EDAD"], errors="coerce")

df_vio.loc[df_vio["VIC_EDAD"] == 999, "VIC_EDAD"] = pd.NA
df_vio.loc[df_vio["AGR_EDAD"] == 999, "AGR_EDAD"] = pd.NA

df_vio = df_vio[(df_vio["VIC_EDAD"].between(10, 110)) | (df_vio["VIC_EDAD"].isna())]
df_vio = df_vio[(df_vio["AGR_EDAD"].between(10, 110)) | (df_vio["AGR_EDAD"].isna())]

ESTANDARIZAR TEXTO

In [5]:
import unicodedata

def limpiar_texto(texto):
    if isinstance(texto, str):
        texto = texto.strip()
        texto = texto.upper()
        texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    return texto

columnas_texto = [
    "DEPTOMCPIO", "AREA", "VIC_SEXO", "VIC_EST_CIV",
    "VIC_ESCOLARIDAD", "VIC_REL_AGR",
    "AGR_SEXO", "AGR_GRUPET", "TIPAGRE",
    "LEY_APLICABLE", "RECUR_DENUN"
]

for col in columnas_texto:
    df_vio[col] = df_vio[col].apply(limpiar_texto)

SUSTITUIR "IGNORADO" POR NULL

In [6]:
df_vio = df_vio.replace("IGNORADO", pd.NA)

print(df_vio.isna().sum())

ANO                   999
MES                     0
DIA                   999
DEPTOMCPIO            732
AREA                10711
VIC_SEXO                0
VIC_EDAD             3079
VIC_EST_CIV         33226
VIC_ESCOLARIDAD      4337
VIC_REL_AGR             0
TOTAL_HIJOS         70195
AGR_SEXO                0
AGR_EDAD            15286
AGR_GRUPET           5217
TIPAGRE                 0
LEY_APLICABLE      192780
RECUR_DENUN         11324
dtype: int64


In [10]:
df_vio.loc[
    df_vio["VIC_ESCOLARIDAD"].str.fullmatch(r"\d+(\.0)?", na=False),
    "VIC_ESCOLARIDAD"
] = pd.NA
print(df_vio["VIC_ESCOLARIDAD"].value_counts(dropna=False))

VIC_ESCOLARIDAD
<NA>                                  231216
NINGUNA                                52641
PRIMARIA GRADO IGNORADO                13351
SEXTO PRIMARIA                         12522
DIVESIFICADO GRADO IGNORADO             6351
SEXTO DIVERSIFICADO                     5845
TERCERO BSICO                           5633
TERCERO PRIMARIA                        4618
BSICO GRADO IGNORADO                    4096
SEGUNDO PRIMARIA                        2909
CUARTO PRIMARIA                         2474
QUINTO PRIMARIA                         1957
QUINTO DIVERSIFICADO                    1783
IGNORADO AO Y NIVEL DE ESCOLARIDAD      1766
SEGUNDO BSICO                           1722
PRIMERO PRIMARIA                        1438
GRADO UNIVERSITARIO IGNORADO            1391
PRIMERO BSICO                           1191
CUARTO DIVERSIFICADO                     854
NaN                                      528
QUINTO AO UNIVERSITARIO                  327
TERCER AO UNIVERSITARIO                

In [11]:
df_vio.to_csv("violencia_intrafamiliar_limpio1.csv", index=False, encoding="utf-8")